In [ ]:
import json
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [ ]:
# Main Kinds
mainkind = ['distribution','plot','numric','query']

# Sub kinds
Dist = ['normal',
        'standard_normal', 'long_tailed', 't', 'f', 'binomial',
        'poisson', 'exponential', 'weibull', 'chi_square']

plot = ['hist', 'box', 'kde', 'pie', 'bar','scatter', 'hex', 'contour', 'violin']

numric = ['mean',
          'w_mean', 't_mean', 'mean_ad', 'median','w_median',
          'median_ad','outlier','std','var','range',
          'iqr','frequency_table','mode','standard_error','percentile',
          'corr_matrix','corr_coff','contingency_table','size','ci']

query = ['select','where','orderby','groupby']

In [ ]:
with open("/content/train_dev.json") as o:
        data = json.load(o)

with open("/content/tables.json") as o:
        tables = json.load(o)

samplesWithAST = []

for e in data:
  function = e['function']
  ast = []

  ### fill the main kind
  index = 0
  for mkind in mainkind:
    if mkind == function[0]:
        ast.append(index)
        break
    index = index + 1
  
  ### fill the sub kind
  if function[0] == 'distribution':
    index = 0
    ast.append([])
    for skind in Dist:
      for oneKind in function[1]:
        if skind == oneKind:
          ast[1].append(index)
      index = index + 1
  elif function[0] == 'plot':
    index = 0
    ast.append([])
    for skind in plot:
      for oneKind in function[1]:
        if skind == oneKind:
          ast[1].append(index)
      index = index + 1
  elif function[0] == 'numric':
    index = 0
    ast.append([])
    for skind in numric:
      for oneKind in function[1]:
        if skind == oneKind:
          ast[1].append(index)
      index = index + 1
  elif function[0] == 'query':
    index = 0
    ast.append([])
    for skind in query:
      for oneKind in function[1]:
        if skind == oneKind:
          ast[1].append(index)
      index = index + 1


  ### fill the tables indexes and columns indexes  
  for db in tables:
    if db['db_id'] == e['db_id']:
      index = 0
      for t in db['table_names_original']:
        if t == function[2]:
           ast.append(index)
           ast.append([])
           functionIndex = 0           
           for cols in function[3]:
              ast[3].append([])
              for c in cols:
                colIndex = 0
                for dbCol in db['column_names_original']:
                  if dbCol[1] == c:
                     if dbCol[0] == index:
                        ast[3][functionIndex].append(colIndex)
                  colIndex = colIndex + 1
              functionIndex = functionIndex + 1
        index = index + 1


  orderbyEP = ['asc','desc'] 
  groupbuEP = ['min','max','count','sum'] 
  whereEP = ['=','>','<','>=','<=','!='] 

  ast.append([])
  paramIndex = 0
  for kindIndex in ast[1]:
    ast[4].append([])
    if function[4][paramIndex]:
      ast[4][paramIndex].append(kindIndex)
      ast[4][paramIndex].append(0)
      ast[4][paramIndex].append(function[4][paramIndex][2])
      
      if function[1][paramIndex] == 'where':
         whereIndex = 0
         for i in whereEP:
           if i == function[4][paramIndex][1]:
             ast[4][paramIndex][1] = whereIndex
           whereIndex = whereIndex + 1

      elif function[1][paramIndex] == 'orderby':
         orderbyIndex = 0
         for i in orderbyEP:
           if i == function[4][paramIndex][2]:
             ast[4][paramIndex][2] = orderbyIndex
           orderbyIndex = orderbyIndex + 1

      elif function[1][paramIndex] == 'groupby':
         groupbyIndex = 0
         for i in groupbuEP:
           if i == function[4][paramIndex][2]:
             ast[4][paramIndex][2] = groupbyIndex
           groupbyIndex = groupbyIndex + 1

      elif function[1][paramIndex] == 'f':
         ast[4][paramIndex].append(kindIndex)
         ast[4][paramIndex].append(0)
         ast[4][paramIndex].append(function[4][paramIndex][5])        


           

    paramIndex = paramIndex + 1

  e['python'] = ast
  samplesWithAST.append(e)
  



    

In [ ]:
with open('train_dev_with_ast.json', 'wt') as data:
   json.dump(samplesWithAST, data, sort_keys=False, indent=4, separators=(',', ': '))